<a href="https://colab.research.google.com/github/haberlmatt/cdeep3m-colab/blob/master/CDeep3M_V2_PredictionGUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='#000FF'> **CDeep3M2:** </font>

In [0]:
#@title # <==== **CDeep3M Colab -- Installation**
#@markdown (Time est.: 20-30min)

#@markdown A hardware check will initially performed to ensure the 
#@markdown runtime envirnment has sufficient GPU vRAM, 
#@markdown otherwise please click: Runtime -> Factory Reset

#################
# Prep
#################

!mkdir -p /home/
# Get the repo
%cd /home/
!git clone https://github.com/haberlmatt/cdeep3m-colab.git

# Install CDeep3M - Packaged into a installer file
!chmod 777 /home/cdeep3m-colab/*

# Make sure enough GPU memory available on this instance
!python /home/cdeep3m-colab/tests/test_GPUmem.py
#!nvidia-smi

##################################
# Run CDeep3M installation
##################################
!/bin/bash /home/cdeep3m-colab/CDeep3M_V2_colab-installer.sh > /home/CDeep3m_installation.log

#################
# Set Environment
#################
import os
os.environ['PATH'] += ":/home/nd_sense/caffe_nd_sense_segmentation/"
os.environ['CAFFE_PATH'] = "/home/nd_sense/caffe_nd_sense_segmentation/"
os.environ['PYTHONPATH'] = "/home/nd_sense/caffe_nd_sense_segmentation/distribute/python/"
os.environ['PATH'] += ":/home/cdeep3m/"

print("CDeep3M installation completed \n   Runnning CDeep3M Version")
!cat /home/cdeep3m/VERSION
print("\nPlease see https://github.com/haberlmatt/cdeep3m-colab for further information on using CDeep3M")

In [0]:
#@title # CDeep3M Colab -- Prediction GUI
#@markdown Simple user interface to run CDeep3M on Colab
#@markdown Please make sure to run installation above first

#@markdown ### Please pick a DOI from the CDeep3M model zoo
#@markdown #### See: http://www.cellimagelibrary.org/cdeep3m
Model_DOI = 'https://doi.org/10.7295/W9CDEEP3M50687' #@param {type:"string"}
#@markdown ### Insert Path to your images here:
Image_Path = '/home/cdeep3m/mito_testsample/testset' #@param {type:"string"}
#@markdown ### Select parameter to run CDeep3M prediction:
Augspeed = '10' #@param ["10", "4", "2", "1"]

selected_aug=int(Augspeed)

run_1fm = False #@param {type:"boolean"}
run_3fm = True #@param {type:"boolean"}
run_5fm = False #@param {type:"boolean"}

#@markdown ---
#@markdown ### Enter a path where files are written:
output_path = "/home/predictout/" #@param {type:"string"}
#@markdown ---

import os
if Model_DOI is not None and Model_DOI !='':
    %cd /home/    
    !rm -r /home/trainedmodel
    !mkdir /home/trainedmodel/
    cmd=str("wget -O model_download " + Model_DOI)
    print(cmd)   
    !{cmd}
    !tar -xvf model_download -C /home/trainedmodel/
    d = '/home/trainedmodel/'
    model_path = [os.path.join(d, o) for o in os.listdir(d) 
                        if os.path.isdir(os.path.join(d,o))]
else:
    print('Provide a model')

if Image_Path is not None and Image_Path !='':
    print('Reading image')
else:
    print('Provide a image path')


# Convert into 1fm, 3fm and 5fm selection into string
fm_list = [str('1fm' * run_1fm), str('3fm' * run_3fm), str('5fm' * run_5fm)]
filtered_fm = filter(lambda x: x != "", fm_list)
combined_fm = list(filtered_fm)
fm_string = ','.join(combined_fm)
if fm_string is not None and fm_string !='':
    print('Applying: ' + fm_string)
else:
    print('Provide a selection for which model depth to run')

print('Starting to run:')
exec_command=str('runprediction.sh --augspeed ' + Augspeed + ' --models ' + fm_string + ' ' + model_path[0] + ' ' + Image_Path + ' ' + output_path) 
print(exec_command)

!{exec_command}

---
---
# **Quick instructions:**

***Most important commands:***
`runprediction.sh`, `PreprocessTraining.py`, `runtraining.sh`

***Hint:***
Upload your data to the `/home/data` directory and start processing



---
---


Please see https://github.com/haberlmatt/cdeep3m-colab for more detailed information on using CDeep3M
Quick examples are provided below

In [0]:
###########################################
# Insert your CLI code here to run CDeep3M:
###########################################

!runprediction.sh ...


# ** Examples and Demoruns**
**Example 1: Perform prediction on demodata using a pretrained model from database**

* `!wget https://doi.org/10.7295/W9CDEEP3M50687`
* `!mkdir /home/trainedmodel/`
* `!tar -xvf W9CDEEP3M50687 -C /home/trainedmodel/`
* `!runprediction.sh --augspeed 10 --models 3fm /home/trainedmodel/membranes_SBEMTEM_denoised_lastmodel /home/6283/ /home/predictout_out/`

**Example 2: Run a training from scratch using demodata**

* `!PreprocessTrainingData.py $cdeep3m_folder/mito_testsample/training/images/ $cdeep3m_folder/mito_testsample/training/labels/ $cdeep3m_folder/mito_testsample/mito_test_augmented`

* `!runtraining.sh --numiterations 50 --snapshot_interval 10 $cdeep3m_folder/mito_testsample/mito_test_augmented $cdeep3m_folder/mito_testsample/demo_trained`

* `$cdeep3m_folder/runprediction.sh $cdeep3m_folder/mito_testsample/demo_trained/ $cdeep3m_folder/mito_testsample/testset/ $cdeep3m_folder/mito_testsample/demo_predictout`

* `!python3 PlotValidation.py $cdeep3m_folder/mito_testsample/demo_predictout/3fm/log/`